In [ ]:
from datascience import *
%matplotlib inline
path_data = '../../data/'
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import math
import numpy as np
from scipy import stats

In [ ]:

colors = Table.read_table(path_data + 'roulette_wheel.csv').column('Color')
pockets = make_array('0','00')
for i in np.arange(1, 37):
    pockets = np.append(pockets, str(i)) 

wheel = Table().with_columns(
    'Pocket', pockets,
    'Color', colors
)

### Le théorème de la limite centrale ###
Très peu des histogrammes de données que nous avons vus en classe ont la forme d'une cloche. Lorsque nous sommes tombés sur une distribution en forme de cloche, il s'agissait presque toujours d'un histogramme empirique d'une statistique basée sur un échantillon aléatoire.

Les exemples ci-dessous montrent deux situations très différentes dans lesquelles une forme approximative en cloche apparaît dans de tels histogrammes.


### Gain net à la roulette ###
La cloche est apparue comme la forme approximative de la somme totale d'argent que nous gagnerions si nous placions le même pari à plusieurs reprises sur différents tours de roulette. 

In [ ]:
wheel

Rappelez-vous que le pari sur le rouge rapporte même de l'argent, 1 pour 1. Nous avons défini la fonction "red_winnings" qui renvoie les gains nets sur un pari de 1 dollar sur le rouge. Plus précisément, la fonction prend une couleur comme argument et renvoie 1 si la couleur est rouge. Pour toutes les autres couleurs, elle renvoie -1.

In [ ]:
def red_winnings(color):
    if color == 'red':
        return 1
    else:
        return -1

Le tableau "red" indique les gains de chaque poche sur le rouge.

In [ ]:
red = wheel.with_column(
    'Winnings: Red', wheel.apply(red_winnings, 'Color')
    )
red

Votre gain net sur un pari est un tirage au sort parmi les "Winnings: Red". Il y a 18/38 chances de gagner \$1, et 20/38 chances de gagner -1 dollar. Cette distribution de probabilité est indiquée dans l'histogramme ci-dessous.

In [ ]:
red.select('Winnings: Red').hist(bins=np.arange(-1.5, 1.6, 1))

Supposons maintenant que vous pariez plusieurs fois sur le rouge. Vos gains nets seront la somme de plusieurs tirages effectués au hasard avec remplacement à partir de la répartition ci-dessus.

Il vous faudra un peu de calcul pour lister toutes les valeurs possibles de vos gains nets ainsi que toutes leurs chances. Nous ne ferons pas cela ; nous allons plutôt approximer la distribution des probabilités par simulation.

Le code ci-dessous simule votre gain net si vous misez \$1 sur le rouge sur 400 tours différents de la roue de la roulette. 


In [ ]:
num_bets = 400
repetitions = 10000

net_gain_red = make_array()

for i in np.arange(repetitions):
    spins = red.sample(num_bets)
    new_net_gain_red = spins.column('Winnings: Red').sum()
    net_gain_red = np.append(net_gain_red, new_net_gain_red)


results = Table().with_column(
    'Net Gain on Red', net_gain_red
    )

In [ ]:
results.hist(bins=np.arange(-80, 50, 6))

C'est un histogramme en forme de cloche, même si la distribution que nous dessinons est loin d'être en forme de cloche.

La distribution est centrée autour de -20 dollars, en gros. Pour voir pourquoi, notez que vos gains seront de 1 dollar sur environ 18/38 des paris, et de 1 dollar  sur les 20/38 restants. Ainsi, vos gains moyens par dollar misé seront d'environ -5,26 cents :

In [ ]:
average_per_bet = 1*(18/38) + (-1)*(20/38)
average_per_bet

Ainsi, en 400 paris, vous vous attendez à ce que votre gain net soit d'environ -21 dollars:

In [ ]:
400 * average_per_bet

Pour confirmation, nous pouvons calculer la moyenne des 10 000 gains nets simulés :

In [ ]:
np.mean(results.column(0))

Regardez la courbe à partir du centre et remarquez que le point d'inflexion est proche de 0. Sur une courbe en forme de cloche, l'écart-type est la distance entre le centre et un point d'inflexion. Le centre est à peu près -20 dollars, ce qui signifie que l'écart-type de la distribution est d'environ 20 dollars.

Dans la section suivante, nous verrons d'où vient la valeur de 20 dollars. Pour l'instant, confirmons notre observation en calculant simplement l'écart-type des 10 000 gains nets simulés:


In [ ]:
np.std(results.column(0))

Sommaire: Le gain net en 400 paris est la somme des 400 montants gagnés sur chaque pari individuel. La distribution de probabilité de cette somme est à peu près normale, avec une moyenne et un écart-type que l'on peut approximer.

### Délai moyen de vol ###
Le tableau "United" contient des données sur les retards au départ de 13 825 vols intérieurs de United Airlines au départ de l'aéroport de San Francisco pendant l'été 2015. La répartition des retards a une longue queue à droite.

In [ ]:
united = Table.read_table(path_data + 'united_summer2015.csv')

In [ ]:
united.select('Delay').hist(bins=np.arange(-20, 300, 10))

Le délai moyen était d'environ 16,6 minutes et l'écart type de 39,5 minutes. Notez l'importance de l'écart type par rapport à la moyenne. Ces grands écarts à droite ont un effet, même s'ils ne représentent qu'une très faible proportion des données.

In [ ]:
mean_delay = np.mean(united.column('Delay'))
sd_delay = np.std(united.column('Delay'))

mean_delay, sd_delay

Supposons maintenant que nous ayons échantillonné 400 retards au hasard avec remplacement. Vous pourriez échantillonner sans remplacement si vous le souhaitez, mais les résultats seraient très similaires à ceux d'un échantillonnage avec remplacement. Si vous échantillonnez quelques centaines de personnes sur 13 825 sans remplacement, vous ne changez guère la population chaque fois que vous retirez une valeur.

Dans l'échantillon, quel pourrait être le délai moyen ? Nous nous attendons à ce qu'il se situe autour de 16 ou 17, car c'est la moyenne de la population ; mais il est probable qu'il soit quelque peu décalé. Voyons ce que nous obtenons par l'échantillonnage. Nous allons travailler avec le tableau "delay" qui ne contient que la colonne des délais.

In [ ]:
delay = united.select('Delay')

In [ ]:
np.mean(delay.sample(400).column('Delay'))

La moyenne de l'échantillon varie en fonction de la façon dont l'échantillon est extrait, nous allons donc simuler le processus d'échantillonnage de façon répétée et dessiner l'histogramme empirique de la moyenne de l'échantillon. Ce sera une approximation de l'histogramme de probabilité de la moyenne de l'échantillon.

In [ ]:
sample_size = 400
repetitions = 10000

means = make_array()

for i in np.arange(repetitions):
    sample = delay.sample(sample_size)
    new_mean = np.mean(sample.column('Delay'))
    means = np.append(means, new_mean)

results = Table().with_column(
    'Sample Mean', means
)

In [ ]:
results.hist(bins=np.arange(10, 25, 0.5))

Une fois de plus, nous voyons une forme de cloche grossière, même si nous tirons d'une distribution très asymétrique. La cloche est centrée quelque part entre 16 et 17, comme nous le prévoyons.

### Théorème de la limite centrale ###

La raison pour laquelle la forme en cloche apparaît dans de tels environnements est un résultat remarquable de la théorie des probabilités appelée **Théorème de la limite centrale**. 

Le **Théorème de la limite centrale** indique que la distribution de probabilité de la somme ou de la moyenne d'un grand échantillon aléatoire tiré avec remplacement sera à peu près normale, peu importe la distribution de la population dans laquelle l'échantillon est tiré.

Comme nous l'avons noté lorsque nous avons étudié les limites de Tchebychev, les résultats qui peuvent être appliqués aux échantillons aléatoires *indépendamment de la distribution de la population* sont très puissants, car en science des données, nous connaissons rarement la distribution de la population.

Le théorème de la limite centrale permet de faire des inférences avec très peu de connaissances sur la population, à condition que nous disposions d'un grand échantillon aléatoire. C'est pourquoi il est central dans le domaine de l'inférence statistique.

### Proportion de fleurs violettes ###
Rappelez-vous le modèle de probabilité de Mendel pour les couleurs des fleurs d'une espèce de pois. Le modèle dit que les couleurs des fleurs des plantes sont comme des tirages au sort avec remplacement de {Violet, Violet, Violet, Blanc}.

Dans un grand échantillon de plantes, quelle proportion environ aura des fleurs violettes ? Nous nous attendons à ce que la réponse soit d'environ 0,75, la proportion de violet dans le modèle. Et, comme les proportions sont des moyennes, le Théorème des limites centrales indique que la distribution de la proportion de plantes violettes dans l'échantillon est à peu près normale.

Nous pouvons le confirmer par simulation. Simulons la proportion de plantes à fleurs violettes dans un échantillon de 200 plantes.

In [ ]:
colors = make_array('Purple', 'Purple', 'Purple', 'White')

model = Table().with_column('Color', colors)

model

In [ ]:
props = make_array()

num_plants = 200
repetitions = 10000

for i in np.arange(repetitions):
    sample = model.sample(num_plants)
    new_prop = np.count_nonzero(sample.column('Color') == 'Purple')/num_plants
    props = np.append(props, new_prop)
    
results = Table().with_column('Sample Proportion: 200', props)

In [ ]:
results.hist(bins=np.arange(0.65, 0.85, 0.01))

Il y a de nouveau cette courbe normale, comme le prédit le Théorème de la limite centrale, centrée autour de 0,75, comme on peut s'y attendre.

Comment cette distribution changerait-elle si nous augmentions la taille de l'échantillon ? Exécutons à nouveau le code avec un échantillon de 800, et rassemblons les résultats des simulations dans le même tableau dans lequel nous avons rassemblé les simulations basées sur une taille d'échantillon de 200. Nous conserverons le même nombre de "répétitions" qu'auparavant afin que les deux colonnes aient la même longueur.

In [ ]:
props2 = make_array()

num_plants = 800

for i in np.arange(repetitions):
    sample = model.sample(num_plants)
    new_prop = np.count_nonzero(sample.column('Color') == 'Purple')/num_plants
    props2 = np.append(props2, new_prop)
    
results = results.with_column('Sample Proportion: 800', props2)

In [ ]:
results.hist(bins=np.arange(0.65, 0.85, 0.01))

Les deux distributions sont à peu près normales, mais l'une est plus étroite que l'autre. Les proportions basées sur un échantillon de 800 sont plus étroitement regroupées autour de 0,75 que celles d'un échantillon de 200. L'augmentation de la taille de l'échantillon a réduit la variabilité de la proportion de l'échantillon.

Cela ne devrait pas être surprenant.  Nous nous sommes souvent appuyés sur l'intuition qu'une taille d'échantillon plus importante réduit généralement la variabilité d'une statistique.  Cependant, dans le cas d'un échantillon moyen, nous pouvons *quantifier* la relation entre la taille de l'échantillon et la variabilité.